In [ ]:
!pip install azure-storage-blob


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.0/407.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.1/207.1 kB 15.7 MB/s eta 0:00:00


In [ ]:
import os
import json
from azure.storage.blob import BlobServiceClient, ContentSettings

# Set Azure connection string and container name
AZURE_CONNECTION_STRING = ""
CONTAINER_NAME = "soap-notes"

# Initialize Blob service and container client
blob_service_client = BlobServiceClient.from_connection_string(AZURE_CONNECTION_STRING)
container_client = blob_service_client.get_container_client(CONTAINER_NAME)

# Function to upload patient SOAP notes with assigned IDs
def upload_json_with_patient_ids(local_folder_path):
    try:
        # Get all JSON files in the folder, sorted alphabetically
        json_files = sorted([f for f in os.listdir(local_folder_path) if f.endswith(".json")])

        for i, filename in enumerate(json_files):
            local_path = os.path.join(local_folder_path, filename)
            patient_id = f"{i+1:03}"  # e.g., '001', '002', '003', ...

            # Read and load the original JSON content
            with open(local_path, "r") as f:
                json_data = json.load(f)

            # Inject patientId field
            json_data["patientId"] = patient_id

            # Convert JSON back to a string for upload
            updated_json_str = json.dumps(json_data, indent=2)

            # Construct blob filename (e.g., patient_001.json)
            blob_name = f"patient_{patient_id}.json"
            blob_client = container_client.get_blob_client(blob=blob_name)

            # Upload the modified content
            blob_client.upload_blob(
                updated_json_str,
                overwrite=True,
                content_settings=ContentSettings(content_type="application/json")
            )

            print(f"✅ Uploaded: {blob_name} with patientId: {patient_id}")

    except Exception as e:
        print(f"❌ Error: {e}")


In [ ]:
upload_json_with_patient_ids("/content/soap-jsons")


✅ Uploaded: patient_001.json with patientId: 001
✅ Uploaded: patient_002.json with patientId: 002
✅ Uploaded: patient_003.json with patientId: 003
✅ Uploaded: patient_004.json with patientId: 004
✅ Uploaded: patient_005.json with patientId: 005
✅ Uploaded: patient_006.json with patientId: 006
✅ Uploaded: patient_007.json with patientId: 007
✅ Uploaded: patient_008.json with patientId: 008
✅ Uploaded: patient_009.json with patientId: 009
✅ Uploaded: patient_010.json with patientId: 010
✅ Uploaded: patient_011.json with patientId: 011
✅ Uploaded: patient_012.json with patientId: 012
✅ Uploaded: patient_013.json with patientId: 013
✅ Uploaded: patient_014.json with patientId: 014
✅ Uploaded: patient_015.json with patientId: 015


In [ ]:
def verify_uploaded_patient_blob(patient_id):
    try:
        blob_name = f"patient_{patient_id}.json"  # e.g., 'patient_001.json'
        blob_client = container_client.get_blob_client(blob=blob_name)

        # Download blob content
        blob_data = blob_client.download_blob().readall()
        json_data = json.loads(blob_data)

        print(f"\n✅ Contents of '{blob_name}':")
        print(json.dumps(json_data, indent=2))

    except Exception as e:
        print(f"❌ Error reading blob {blob_name}: {e}")


In [ ]:
# Check contents of patient_001.json
verify_uploaded_patient_blob("001")


✅ Contents of 'patient_001.json':
{
  "Patient demographics": {
    "Age": 35,
    "Gender": "Female",
    "Ethnicity": "Caucasian",
    "Occupation": "Teacher",
    "Education": "Bachelor's degree",
    "Marital status": "Married"
  },
  "Chief complaint": {
    "Description of the chief complaint": "Abdominal pain and bloating",
    "Duration of symptoms": "2 days",
    "Associated symptoms": "Nausea, vomiting, and diarrhea"
  },
  "Relevant past medical history": {
    "Allergies": "None",
    "Medical conditions": [
      "Hypertension",
      "Osteoarthritis"
    ],
    "Previous hospitalizations": "None",
    "Surgeries": "None",
    "Family history of significant diseases": "None"
  },
  "Social history": {
    "Smoking status": "Non-smoker",
    "Alcohol consumption": "Moderate",
    "Recreational drug use": "None",
    "Family history of lung disease or other relevant conditions": "None",
    "Travel history": "None"
  },
  "Review of systems": "The patient reports a history 

Function to Read All JSON Files from Azure Blob Storage



In [ ]:
def read_all_soap_notes():
    soap_notes = []

    try:
        print(f"Reading all JSON files from container '{CONTAINER_NAME}'...")

        # List blobs in container
        for blob in container_client.list_blobs():
            if blob.name.endswith(".json"):
                blob_client = container_client.get_blob_client(blob)

                # Download and decode the JSON
                blob_data = blob_client.download_blob().readall()
                note = json.loads(blob_data)
                soap_notes.append(note)

                print(f"✅ Loaded: {blob.name}")

    except Exception as e:
        print(f"Error reading from Azure Blob Storage: {e}")

    return soap_notes

In [ ]:
all_notes = read_all_soap_notes()

print(f"\n📄 Total notes retrieved: {len(all_notes)}")
if all_notes:
    print(json.dumps(all_notes[0], indent=2))  # Preview the first one


Reading all JSON files from container 'soap-notes'...
✅ Loaded: patient_001.json
✅ Loaded: patient_002.json
✅ Loaded: patient_003.json
✅ Loaded: patient_004.json
✅ Loaded: patient_005.json
✅ Loaded: patient_006.json
✅ Loaded: patient_007.json
✅ Loaded: patient_008.json
✅ Loaded: patient_009.json
✅ Loaded: patient_010.json
✅ Loaded: patient_011.json
✅ Loaded: patient_012.json
✅ Loaded: patient_013.json
✅ Loaded: patient_014.json
✅ Loaded: patient_015.json

📄 Total notes retrieved: 15
{
  "Patient demographics": {
    "Age": 35,
    "Gender": "Female",
    "Ethnicity": "Caucasian",
    "Occupation": "Teacher",
    "Education": "Bachelor's degree",
    "Marital status": "Married"
  },
  "Chief complaint": {
    "Description of the chief complaint": "Abdominal pain and bloating",
    "Duration of symptoms": "2 days",
    "Associated symptoms": "Nausea, vomiting, and diarrhea"
  },
  "Relevant past medical history": {
    "Allergies": "None",
    "Medical conditions": [
      "Hypertension"

Read a Single File by Name

In [ ]:
def read_soap_note_by_filename(blob_name):
    try:
        blob_client = container_client.get_blob_client(blob_name)
        blob_data = blob_client.download_blob().readall()
        return json.loads(blob_data)

    except Exception as e:
        print(f"❌ Error: {e}")
        return None


In [ ]:
note = read_soap_note_by_filename("patient_002.json")
print(json.dumps(note, indent=2))


{
  "Patient demographics": {
    "Age": 45,
    "Gender": "Female",
    "Ethnicity": "Caucasian",
    "Occupation": "Teacher",
    "Education": "Bachelor's degree",
    "Marital status": "Married"
  },
  "Chief complaint": {
    "Description of the chief complaint": "Shortness of breath and chest tightness",
    "Duration of symptoms": "1 week",
    "Associated symptoms": "Cough, fatigue, difficulty sleeping"
  },
  "Relevant past medical history": {
    "Allergies": "None",
    "Medical conditions": [
      "Hypertension",
      "Diabetes",
      "Osteoarthritis"
    ],
    "Previous hospitalizations": "None",
    "Surgeries": "None",
    "Family history of significant diseases": "None"
  },
  "Social history": {
    "Smoking status": "Non-smoker",
    "Alcohol consumption": "Moderate",
    "Recreational drug use": "None",
    "Family history of lung disease or other relevant conditions": "None",
    "Travel history": "None"
  },
  "Review of systems": "Patient reports a history of h

Single function

In [ ]:
import os
import json
from azure.storage.blob import BlobServiceClient, ContentSettings

# Set Azure connection string and container name
AZURE_CONNECTION_STRING = ""
CONTAINER_NAME = "soap-notes"

# Initialize Azure blob clients
blob_service_client = BlobServiceClient.from_connection_string(AZURE_CONNECTION_STRING)
container_client = blob_service_client.get_container_client(CONTAINER_NAME)

def handle_blob_operations(mode, local_folder_path=None, patient_id=None, blob_filename=None):
    try:
        if mode == "upload":
            if not local_folder_path:
                print("❗ Please provide a local folder path.")
                return
            json_files = sorted([f for f in os.listdir(local_folder_path) if f.endswith(".json")])
            for i, filename in enumerate(json_files):
                patient_id = f"{i+1:03}"
                local_path = os.path.join(local_folder_path, filename)
                with open(local_path, "r") as f:
                    json_data = json.load(f)
                json_data["patientId"] = patient_id
                updated_json_str = json.dumps(json_data, indent=2)
                blob_name = f"patient_{patient_id}.json"
                blob_client = container_client.get_blob_client(blob_name)
                blob_client.upload_blob(
                    updated_json_str,
                    overwrite=True,
                    content_settings=ContentSettings(content_type="application/json")
                )
                print(f"✅ Uploaded: {blob_name} with patientId: {patient_id}")

        elif mode == "verify":
            if not patient_id:
                print("❗ Please provide a patient ID to verify.")
                return
            blob_name = f"patient_{patient_id}.json"
            blob_client = container_client.get_blob_client(blob_name)
            blob_data = blob_client.download_blob().readall()
            json_data = json.loads(blob_data)
            print(f"\n✅ Contents of '{blob_name}':")
            print(json.dumps(json_data, indent=2))

        elif mode == "read_all":
            soap_notes = []
            print(f"\n📁 Reading all JSON files from container '{CONTAINER_NAME}'...")
            for blob in container_client.list_blobs():
                if blob.name.endswith(".json"):
                    blob_client = container_client.get_blob_client(blob)
                    blob_data = blob_client.download_blob().readall()
                    note = json.loads(blob_data)
                    soap_notes.append(note)
                    print(f"✅ Loaded: {blob.name}")
            print(f"\n📄 Total notes retrieved: {len(soap_notes)}")
            if soap_notes:
                print(json.dumps(soap_notes[0], indent=2))  # preview first one

        elif mode == "read_one":
            if not blob_filename:
                print("❗ Please provide a blob filename to read.")
                return
            blob_client = container_client.get_blob_client(blob_filename)
            blob_data = blob_client.download_blob().readall()
            note = json.loads(blob_data)
            print(json.dumps(note, indent=2))

        else:
            print("❌ Invalid mode. Choose from: 'upload', 'verify', 'read_all', 'read_one'.")

    except Exception as e:
        print(f"❌ Error: {e}")


Upload function

In [ ]:
handle_blob_operations("upload", local_folder_path="/content/soap-jsons")


Verify function

In [ ]:
handle_blob_operations("verify", patient_id="002")


Read All function

In [ ]:
handle_blob_operations("read_all")


Read specific function

In [ ]:
handle_blob_operations("read_one", blob_filename="patient_003.json")


#Azure OpenAI with Blob storage Implementation for Patient Summary Generation

In [ ]:
!pip install flask langchain openai azure-storage-blob pyngrok --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.0/407.0 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.1/207.1 kB 12.9 MB/s eta 0:00:00


In [ ]:
!pip uninstall -y langchain langchain-core langchain-community langchain-openai
!pip install langchain-core==0.1.32 langchain-community==0.0.26 langchain-openai==0.0.8 langchain --upgrade


Found existing installation: langchain 0.3.23
Uninstalling langchain-0.3.23:
  Successfully uninstalled langchain-0.3.23
Found existing installation: langchain-core 0.3.51
Uninstalling langchain-core-0.3.51:
  Successfully uninstalled langchain-core-0.3.51
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence


In [ ]:
llm = AzureChatOpenAI(
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    deployment_name="your_deployment_name",
    openai_api_version="2023-05-15",  # required version for Azure
    temperature=0.3
)


In [ ]:
import os, json, threading, getpass
from flask import Flask, request, jsonify
from azure.storage.blob import BlobServiceClient
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence

# Prompt for config
os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass("🔐 Azure OpenAI API Key: ")
os.environ["AZURE_OPENAI_ENDPOINT"] = input("🔗 Azure Endpoint (e.g. https://YOUR-RESOURCE.openai.azure.com): ")
DEPLOYMENT_NAME = input("🧠 Azure OpenAI deployment name (e.g. gpt35): ")
AZURE_CONNECTION_STRING = getpass.getpass("📦 Azure Blob Connection String: ")
CONTAINER_NAME = input("📁 Azure Blob Container Name (e.g. soap-notes): ")

# Azure Blob client
blob_service_client = BlobServiceClient.from_connection_string(AZURE_CONNECTION_STRING)
container_client = blob_service_client.get_container_client(CONTAINER_NAME)

# LangChain + Azure OpenAI setup
llm = AzureChatOpenAI(
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    deployment_name=DEPLOYMENT_NAME,
    openai_api_version="2023-05-15",
    temperature=0.3
)

prompt = PromptTemplate.from_template("""
You are a medical assistant. Summarize the following patient SOAP note in 4-5 clear, professional sentences:

{note}
""")
summary_chain: RunnableSequence = prompt | llm | StrOutputParser()

# Flask app
app = Flask(__name__)

def get_patient_note(patient_id):
    blob_name = f"patient_{patient_id}.json"
    blob_client = container_client.get_blob_client(blob_name)
    blob_data = blob_client.download_blob().readall()
    return json.loads(blob_data)

def generate_summary(note_json):
    note_text = json.dumps(note_json, indent=2)
    return summary_chain.invoke({"note": note_text})

@app.route("/summarize", methods=["GET"])
def summarize():
    patient_id = request.args.get("patient_id")
    if not patient_id:
        return jsonify({"error": "Missing patient_id"}), 400
    try:
        note = get_patient_note(patient_id)
        summary = generate_summary(note)
        return jsonify({"patient_id": patient_id, "summary": summary})
    except Exception as e:
        return jsonify({"error": str(e)}), 500


🔐 Azure OpenAI API Key: ··········
🔗 Azure Endpoint (e.g. https://YOUR-RESOURCE.openai.azure.com): https://openai-medistory.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2025-01-01-preview
🧠 Azure OpenAI deployment name (e.g. gpt35): gpt-35-turbo
📦 Azure Blob Connection String: ··········
📁 Azure Blob Container Name (e.g. soap-notes): soap-notes


In [ ]:
import os, getpass, json
from azure.storage.blob import BlobServiceClient
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence

# 🔐 Secure credentials
os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass("🔐 Azure OpenAI API Key: ")
os.environ["AZURE_OPENAI_ENDPOINT"] = input("🔗 Azure Endpoint (e.g. https://YOUR-RESOURCE.openai.azure.com): ")
deployment_name = input("🧠 Azure OpenAI deployment name (e.g. gpt35): ")

AZURE_CONNECTION_STRING = getpass.getpass("🔐 Azure Blob Connection String: ")
CONTAINER_NAME = input("📦 Azure Blob Container Name (e.g. soap-notes): ")
PATIENT_ID = input("🧑‍⚕️ Patient ID (e.g. 002): ")

# 🔌 Azure Blob Access
blob_service_client = BlobServiceClient.from_connection_string(AZURE_CONNECTION_STRING)
container_client = blob_service_client.get_container_client(CONTAINER_NAME)
blob_name = f"patient_{PATIENT_ID}.json"
blob_client = container_client.get_blob_client(blob_name)
blob_data = blob_client.download_blob().readall()
note_json = json.loads(blob_data)

# 🤖 LangChain + Azure OpenAI Setup
llm = AzureChatOpenAI(
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    deployment_name=deployment_name,
    openai_api_version="2023-05-15",
    temperature=0.3
)

prompt = PromptTemplate.from_template("""
You are a medical assistant. Summarize the following patient SOAP note in 4-5 clear, professional sentences:

{note}
""")

chain: RunnableSequence = prompt | llm | StrOutputParser()
summary = chain.invoke({"note": json.dumps(note_json, indent=2)})

# 📋 Output
print(f"\n✅ Summary for patient_{PATIENT_ID}.json:\n\n{summary}")


🔐 Azure OpenAI API Key: ··········
🔗 Azure Endpoint (e.g. https://YOUR-RESOURCE.openai.azure.com): https://hxd23-m9kr43fq-eastus2.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview
🧠 Azure OpenAI deployment name (e.g. gpt35): gpt-4o
🔐 Azure Blob Connection String: ··········
📦 Azure Blob Container Name (e.g. soap-notes): soap-notes
🧑‍⚕️ Patient ID (e.g. 002): 005

✅ Summary for patient_005.json:

A 45-year-old Caucasian female teacher with a history of asthma presented with a chronic cough, shortness of breath, wheezing, chest tightness, and difficulty sleeping for the past three months. Physical examination revealed bilateral wheezing on auscultation and tenderness in the right upper quadrant of the abdomen. Vital signs were stable, though the respiratory rate was elevated at 22 breaths per minute, and oxygen saturation was slightly reduced at 95%. The primary diagnosis is asthma exacerbation with pneumonia, with differential consider